# 📊 Stock Intraday Data Downloader

**Automated system to download 30-minute interval data from Yahoo Finance**

- **Data Sources**: Custom ticker list from `tickers.csv`
- **Interval**: 30 minutes
- **Storage**: Compressed CSV files (`.csv.gz`)
- **Mode**: Incremental append (no overwrites)
- **Logging**: Full error tracking in `download_log.txt`

---

## 1️⃣ Installation & Setup

Install required packages

In [11]:
!pip install yfinance pandas requests -q

## 2️⃣ Import Libraries & Configuration

In [12]:
import yfinance as yf
import pandas as pd
import requests
from datetime import datetime, timedelta
from pathlib import Path
import time
import logging
import warnings

# Suppress yfinance warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', module='yfinance')

# Configure logging (only to file, not console)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('download_log.txt')
    ]
)

# Create data directory
DATA_DIR = Path("stock_data")
DATA_DIR.mkdir(exist_ok=True)

print("✅ Configuration complete")
print(f"📁 Data directory: {DATA_DIR.absolute()}")

✅ Configuration complete
📁 Data directory: c:\Users\trion\OneDrive\Desktop\Files\Stock Prices Dataset\stock_data


## 3️⃣ Load Ticker List from CSV

Russell 1000 Companies as of August 22, 2025, are 1007

In [13]:
# Embedded ticker list (replaces reading tickers.csv).
# The full ticker list is written here so the notebook won't call tickers.csv at runtime.
all_tickers = ["MMM","AOS","AAON","ABT","ABBV","ACHC","ACN","AYI","ADBE","ADT","WMS","AMD",
               "ACM","AES","AMG","AFRM","AFL","AGCO","A","ADC","AGNC","AL","APD","ABNB","AKAM",
               "ALK","ALB","ACI","AA","ARE","ALGN","ALLE","ALGM","LNT","ALSN","ALL","ALLY",
               "ALNY","GOOGL","GOOG","MO","AMZN","AMCR","DOX","AMTM","AS","AEE","AAL","AEP",
               "AXP","AFG","AMH","AIG","AMT","AWK","COLD","AMP","AME","AMGN","AMKR","APH","ADI",
               "AU","NLY","AM","AR","AON","APA","APG","APLS","APO","APPF","AAPL","AIT","AMAT",
               "APP","ATR","APTV","ARMK","ACGL","ADM","ARES","ANET","AWI","ARW","AJG","ASH",
               "AIZ","AGO","ALAB","ASTS","T","ATI","TEAM","ATO","AUR","ADSK","ADP","AN","AZO",
               "AVB","AVTR","AVY","CAR","AVT","AXTA","AXS","AXON","BKR","BALL","BAC","OZK",
               "BBWI","BAX","BDX","BRBR","BSY","BRK-B","BBY","BILL","BIO","TECH","BIIB","BMRN",
               "BIRK","BJ","BLK","BX","HRB","XYZ","OWL","BK","BA","BOKF","BKNG","BAH","BWA",
               "SAM","BSX","BYD","BFAM","BHF","BMY","BRX","AVGO","BR","BAM","BEPC","BRO","BF-A",
               "BF-B","BRKR","BC","BLDR","BG","BURL","BWXT","BXP","CHRW","CACI","CDNS","CZR",
               "CPT","CPB","COF","CAH","CAI","CSL","CG","KMX","CCL","CRS","CARR","CVNA","CAT",
               "CAVA","CBOE","CBRE","CDW","CE","CELH","COR","CNC","CNP","CERT","CF","CRL","SCHW",
               "CHTR","CHE","LNG","CVX","CHWY","CMG","CHH","CHRD","CB","CHD","CHDN","CIEN","CI",
               "CINF","CTAS","CRUS","CSCO","C","CFG","CIVI","CLVT","CLH","CWEN-A","CWEN","CLF",
               "CLX","NET","CME","CMS","CNA","CNH","KO","COKE","CGNX","CTSH","COHR","COIN","CL",
               "COLB","COLM","CMCSA","CMA","FIX","CBSH","CAG","CNXC","CFLT","COP","ED","STZ",
               "CEG","COO","CPRT","CORT","CNM","GLW","CPAY","CTVA","CSGP","COST","CTRA","COTY",
               "CPNG","CUZ","CR","CXT","CACC","CRH","CROX","CRWD","CCI","CCK","CSX","CUBE","CMI",
               "CW","CVS","DHI","DHR","DRI","DAR","DDOG","DVA","DAY","DECK","DE","DAL","DELL",
               "XRAY","DVN","DXCM","FANG","DKS","DLR","DDS","DOCU","DLB","DG","DLTR","D","DPZ",
               "DCI","DASH","DV","DOV","DOW","DOCS","DKNG","DBX","DTM","DTE","DUK","DUOL","DD",
               "BROS","DXC","DT","ELF","EXP","EWBC","EGP","EMN","ETN","EBAY","ECL","EIX","EW",
               "ELAN","ESTC","EA","ESI","ELV","EME","EMR","EHC","ENPH","ENTG","ETR","NVST","EOG",
               "EPAM","EPR","EQT","EFX","EQIX","EQH","ELS","EQR","ESAB","WTRG","ESS","EL","ETSY",
               "EEFT","EVR","EG","EVRG","ES","ECG","EXAS","EXEL","EXC","EXLS","EXE","EXPE","EXPD",
               "EXR","XOM","FFIV","FDS","FICO","FAST","FRT","FDX","FERG","FNF","FIS","FITB","FAF",
               "FCNCA","FHB","FHN","FR","FSLR","FE","FI","FIVE","FLEX","FND","FLO","FLS","FLUT",
               "FMC","FNB","F","FTNT","FTV","FBIN","FOXA","FOX","BEN","FRHC","FCX","FRPT","FYBR",
               "CFR","FTAI","FCN","GME","GLPI","GAP","GRMN","IT","GTES","GLIBA","GLIBK","GE",
               "GEHC","GEV","GEN","GNRC","GD","GIS","GM","G","GNTX","GPC","GILD","GTLB","GPN",
               "GFS","GLOB","GL","GMED","GDDY","GS","GGG","LOPE","GPK","GWRE","GXO","HAL","HALO",
               "HLNE","THG","HOG","HIG","HAS","HAYW","HCA","HR","DOC","HEI-A","HEI","JKHY","HSY",
               "HPE","HXL","DINO","HIW","HLT","HOLX","HD","HON","HRL","HST","HLI","HHH","HWM",
               "HPQ","HUBB","HUBS","HUM","HBAN","HII","HUN","H","IAC","IBM","IDA","IEX","IDXX",
               "ITW","ILMN","INCY","INFA","IR","INGM","INGR","INSM","INSP","PODD","INTC","IBKR",
               "ICE","IFF","IP","IPG","INTU","ISRG","IVZ","INVH","IONS","IPGP","IQV","IRDM",
               "IRM","ITT","JBL","J","JHX","JHG","JAZZ","JBHT","JEF","JNJ","JCI","JLL","JPM",
               "KRMN","KBR","K","KMPR","KVUE","KDP","KEY","KEYS","KRC","KMB","KIM","KMI","KNSL",
               "KEX","KKR","KLAC","KNX","KHC","KR","KD","LHX","LH","LRCX","LAMR","LW","LSTR",
               "LVS","LSCC","LAZ","LEA","LDOS","LEN","LEN-B","LII","DRS","LBRDA","LBRDK","LBTYA",
               "LBTYK","FWONA","FWONK","LLYVA","LLYVK","LNW","LLY","LECO","LNC","LIN","LINE",
               "LAD","LFUS","LYV","LKQ","LOAR","LMT","L","LPX","LOW","LPLA","LCID","LULU","LITE",
               "LYFT","LYB","MTB","MTSI","M","MSGS","MANH","MAN","CART","MPC","MKL","MKTX",
               "MAR","MMC","MLM","MRVL","MAS","MASI","MTZ","MA","MTDR","MTCH","MAT","MKC",
               "MCD","MCK","MDU","MPW","MEDP","MDT","MRK","META","MET","MTD","MTG","MGM",
               "MCHP","MU","MSFT","MSTR","MAA","MIDD","TIGO","MRP","MKSI","MRNA","MHK","MOH",
               "TAP","MDLZ","MDB","MPWR","MNST","MCO","MS","MORN","MOS","MSI","MP","MSA","MSM",
               "MSCI","MLI","MUSA","NDAQ","NTRA","NFG","NSA","NCNO","NTAP","NFLX","NBIX","NYT",
               "NWL","NEU","NEM","NWSA","NWS","NXST","NEE","NIQ","NKE","NI","NNN","NDSN","NSC",
               "NTRS","NOC","NCLH","NOV","NRG","NU","NUE","NTNX","NVT","NVDA","NVR","ORLY","OXY",
               "OGE","OKTA","ODFL","ORI","OLN","OLLI","OHI","OMC","ONON","ON","OMF","OKE","ONTO",
               "ORCL","OGN","OSK","OTIS","OVV","OC","PCAR","PKG","PLTR","PANW","PK","PH","PSN",
               "PAYX","PAYC","PCTY","PYPL","PEGA","PENN","PAG","PNR","PEN","PEP","PFGC","PR",
               "PRGO","PFE","PCG","PM","PSX","PPC","PNFP","PNW","PINS","PLNT","PNC","POOL",
               "BPOP","POST","PPG","PPL","TROW","PRI","PRMB","PFG","PCOR","PG","PGR","PLD",
               "PB","PRU","PTC","PSA","PEG","PHM","PSTG","PVH","QGEN","QRVO","QCOM","PWR","QS",
               "DGX","QXO","RAL","RL","RRC","RJF","RYN","RBA","RBC","O","RDDT","RRX","REG",
               "REGN","RF","RGA","RS","RNR","RGEN","RSG","RMD","QSR","RVMD","RVTY","REXR",
               "REYN","RH","RNG","RITM","RIVN","RLI","RHI","HOOD","RBLX","RKT","RKLB","ROK",
               "ROIV","ROKU","ROL","ROP","ROST","RCL","RGLD","RPRX","RPM","RTX","RBRK","RYAN",
               "R","SPGI","SAIA","SAIL","SAIC","CRM","SLM","IOT","SNDK","SRPT","SBAC","HSIC",
               "SLB","SNDR","SMG","SEB","SEE","SEIC","SRE","ST","S","SCI","NOW","SN","SHW",
               "FOUR","SLGN","SPG","SSD","SIRI","SITE","SWKS","SFD","SJM","SW","SNA","SNOW",
               "SOFI","SOLS","SOLV","SGI","SON","SHC","SO","SCCO","SSB","LUV","SPR","SPOT",
               "SFM","SSNC","STAG","SARO","SWK","SBUX","STWD","STT","STLD","STE","SF","SYK",
               "SMMT","SUI","SMCI","SYF","SNPS","SNV","SYY","TMUS","TTWO","TLN","TPR","TRGP",
               "TGT","SNX","FTI","TDY","TFX","TEM","THC","TDC","TER","TSLA","TTEK","TXN","TPL",
               "TXRH","TXT","TMO","TFSL","THO","TKR","TJX","TKO","TOST","TOL","BLD","TTC",
               "TPG","TSCO","TTD","TW","TT","TDG","TRU","TNL","TRV","TREX","TRMB","TFC",
               "DJT","TWLO","TYL","TSN","UHAL","UHAL-B","USB","UBER","UI","UDR","UGI","PATH",
               "ULTA","RARE","UAA","UA","UNP","UAL","UPS","URI","UTHR","UWMC","UNH","U","OLED",
               "UHS","UNM","USFD","MTN","VLO","VMI","VVV","VEEV","VTR","VLTO","VRSN","VRSK",
               "VZ","VRTX","VRT","VFC","VTRS","VICI","VIK","VKTX","VNOM","VIRT","V","VST",
               "VNT","VNO","VOYA","VMC","WPC","WRB","GWW","WAB","WMT","DIS","WBD","WM","WAT",
               "WSO","W","WFRD","WBS","WEC","WFC","WELL","WEN","WCC","WST","WAL","WDC","WU",
               "WLK","WEX","WY","WHR","WTM","WMB","WSM","WTW","WSC","WING","WTFC","WWD","WDAY",
               "WH","WYNN","XEL","XP","XPO","XYL","YETI","YUM","ZBRA","ZG","Z","ZBH","ZION",
               "ZTS","ZM","GTM","ZS"]
print(f"✅ Loaded {len(all_tickers)} tickers (embedded)")

✅ Loaded 1007 tickers (embedded)


## 4️⃣ Core Functions

### Download Function

In [14]:
def download_ticker_data(ticker, start_date, end_date, interval="30m"):
    """
    Download stock data for a single ticker
    Returns DataFrame with only required columns
    """
    try:
        # Download data from Yahoo Finance
        df = yf.download(
            ticker, 
            start=start_date, 
            end=end_date, 
            interval=interval,
            progress=False  # Disable yfinance progress bar
        )
        
        if df.empty:
            logging.warning(f"No data available for {ticker}")
            return None
        
        # Flatten MultiIndex columns if present
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
        
        # Reset index to make Datetime a column
        df = df.reset_index()
        
        # Add ticker column
        df['Ticker'] = ticker
        
        # Select only required columns
        required_columns = ['Datetime', 'Close', 'High', 'Low', 'Open', 'Volume', 'Ticker']
        df = df[required_columns]
        
        return df
        
    except Exception as e:
        logging.error(f"Error downloading {ticker}: {str(e)}")
        return None


def save_data(ticker, df):
    """
    Save data to compressed CSV file (append mode)
    """
    file_path = DATA_DIR / f"{ticker}.csv.gz"
    
    try:
        # If file exists, load and append new data
        if file_path.exists():
            existing_df = pd.read_csv(file_path, compression='gzip', parse_dates=['Datetime'])
            df = pd.concat([existing_df, df], ignore_index=True)
            # Remove duplicates based on Datetime and Ticker
            df = df.drop_duplicates(subset=['Datetime', 'Ticker'], keep='last')
        
        # Sort by Datetime
        df = df.sort_values('Datetime')
        
        # Save to compressed CSV
        df.to_csv(file_path, compression='gzip', index=False)
        
        return True
    except Exception as e:
        logging.error(f"Error saving data for {ticker}: {str(e)}")
        return False


def download_all(tickers, start_date, end_date, interval="30m", delay=0.2):
    """
    Download data for all tickers with progress bar
    """
    from tqdm import tqdm
    
    successful = 0
    failed = 0
    
    # Progress bar
    for ticker in tqdm(tickers, desc="📥 Downloading", ncols=100, bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]'):
        df = download_ticker_data(ticker, start_date, end_date, interval)
        
        if df is not None and not df.empty:
            if save_data(ticker, df):
                successful += 1
                logging.info(f"✅ {ticker}: {len(df)} records saved")
            else:
                failed += 1
        else:
            failed += 1
            logging.error(f"❌ {ticker}: Failed to download")
        
        # Rate limiting
        time.sleep(delay)
    
    return successful, failed

print("✅ Functions defined successfully")

✅ Functions defined successfully


### Save Function (CSV with compression)

In [15]:
def save_data(df, ticker, append=True):
    """
    Save data to compressed CSV file
    
    Args:
        df (pd.DataFrame): Data to save
        ticker (str): Ticker symbol
        append (bool): If True, append to existing file
    
    Returns:
        bool: True if successful, False otherwise
    """
    try:
        file_path = DATA_DIR / f"{ticker}.csv.gz"
        
        if append and file_path.exists():
            # Read existing data
            existing_df = pd.read_csv(file_path, compression='gzip', parse_dates=['Datetime'])
            
            # Merge with new data
            combined_df = pd.concat([existing_df, df], ignore_index=True)
            
            # Remove duplicates
            combined_df = combined_df.drop_duplicates(subset=['Datetime', 'Ticker'], keep='last')
            
            # Sort by datetime
            combined_df = combined_df.sort_values('Datetime').reset_index(drop=True)
            
            combined_df.to_csv(file_path, compression='gzip', index=False)
            logging.info(f"{ticker}: Added {len(df)} new records (total: {len(combined_df)})")
        else:
            df.to_csv(file_path, compression='gzip', index=False)
            logging.info(f"{ticker}: Saved {len(df)} records")
        
        return True
    
    except Exception as e:
        logging.error(f"{ticker}: Save error - {str(e)}")
        return False

print("✅ Save function defined")

✅ Save function defined


### Batch Download Function

In [16]:
def download_all(tickers, start_date, end_date, interval="30m", delay=0.2):
    """
    Download data for all tickers with error handling
    
    Args:
        tickers (list): List of ticker symbols
        start_date (str): Start date
        end_date (str): End date
        interval (str): Data interval
        delay (float): Delay between requests (seconds)
    
    Returns:
        tuple: (successful_count, failed_count)
    """
    total = len(tickers)
    successful = 0
    failed = 0
    
    logging.info(f"Starting download of {total} tickers...")
    logging.info(f"Period: {start_date} to {end_date}")
    logging.info(f"Interval: {interval}")
    
    for i, ticker in enumerate(tickers, 1):
        logging.info(f"[{i}/{total}] Processing {ticker}...")
        
        # Download data
        df = download_ticker_data(ticker, start_date, end_date, interval)
        
        if df is not None and not df.empty:
            # Save data
            if save_data(df, ticker, append=True):
                successful += 1
            else:
                failed += 1
        else:
            failed += 1
        
        # Delay to avoid rate limiting
        time.sleep(delay)
    
    # Summary
    logging.info("=" * 50)
    logging.info("Download complete!")
    logging.info(f"Successful: {successful}/{total}")
    logging.info(f"Failed: {failed}/{total}")
    logging.info("=" * 50)
    
    return successful, failed

print("✅ Batch download function defined")

✅ Batch download function defined


## 5️⃣ Calculate Date Range

Automatically set to download data from **2 days ago to today** (3 days total)

In [17]:
# Calculate dates (from 2 days ago to today)
# Example: if today is Nov 5, downloads data from Nov 3, 4, and 5
end_date = datetime.now()
start_date = end_date - timedelta(days=2)

# Format 
start_str = start_date.strftime("%Y-%m-%d")
end_str = end_date.strftime("%Y-%m-%d")

print(f"📅 Download period: {start_str} to {end_str}")
print(f"📆 Days included: 3 days (from 2 days ago to today)")
print(f"⏰ Interval: 30 minutes")

📅 Download period: 2025-11-03 to 2025-11-05
📆 Days included: 3 days (from 2 days ago to today)
⏰ Interval: 30 minutes


## 6️⃣ Run Download

### Option A: Test with Sample Tickers (Recommended First)

In [18]:
""" # Test with a small sample
test_tickers = ['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'NVDA', 'BTC-USD', 'ETH-USD', 'BNB-USD']

print(f"🧪 TEST MODE - Downloading {len(test_tickers)} tickers...\n")

successful, failed = download_all(
    tickers=test_tickers,
    start_date=start_str,
    end_date=end_str,
    interval="30m",
    delay=0.2
)

print(f"\n✅ Test complete: {successful} successful, {failed} failed") """

' # Test with a small sample\ntest_tickers = [\'AAPL\', \'MSFT\', \'GOOGL\', \'TSLA\', \'NVDA\', \'BTC-USD\', \'ETH-USD\', \'BNB-USD\']\n\nprint(f"🧪 TEST MODE - Downloading {len(test_tickers)} tickers...\n")\n\nsuccessful, failed = download_all(\n    tickers=test_tickers,\n    start_date=start_str,\n    end_date=end_str,\n    interval="30m",\n    delay=0.2\n)\n\nprint(f"\n✅ Test complete: {successful} successful, {failed} failed") '

### Option B: Full Download (All Tickers)

⚠️ **This will take approximately 6-7 minutes**

In [19]:
# Full download of all tickers
print(f"📥 Starting download of {len(all_tickers)} tickers...")
print(f"⏱️  Estimated time: ~{len(all_tickers) * 0.35 / 60:.1f} minutes\n")

start_time = time.time()

successful, failed = download_all(
    tickers=all_tickers,
    start_date=start_str,
    end_date=end_str,
    interval="30m",
    delay=0.2
)

elapsed_time = time.time() - start_time

print(f"\n⏱️  Total time: {elapsed_time/60:.2f} minutes")
print(f"✅ Successfully downloaded: {successful}/{len(all_tickers)} tickers")
print(f"❌ Failed: {failed}/{len(all_tickers)} tickers")

📥 Starting download of 1007 tickers...
⏱️  Estimated time: ~5.9 minutes


⏱️  Total time: 6.78 minutes
✅ Successfully downloaded: 1007/1007 tickers
❌ Failed: 0/1007 tickers


If some errors occured, no problem: check the download_log.txt

In [20]:
# 🔄 FUNZIONI PER RE-DOWNLOAD TICKERS SPECIFICI

def redownload_tickers(tickers, start_date=None, end_date=None):
    """
    Re-download specifici tickers
    Esempi:
        redownload_tickers('AAPL')
        redownload_tickers(['AAPL', 'MSFT', 'GOOGL'])
    """
    if isinstance(tickers, str):
        tickers = [tickers]
    
    if start_date is None:
        start_date = start_str
    if end_date is None:
        end_date = end_str
    
    print(f"🔄 Re-downloading {len(tickers)} ticker(s)...")
    return download_all(tickers, start_date, end_date, "30m", 0.2)


def get_missing_tickers():
    """Trova ticker senza file"""
    existing = {f.stem for f in DATA_DIR.glob("*.csv.gz")}
    return [t for t in all_tickers if t not in existing]


# ESEMPIO USO:
# redownload_tickers('MDT')
# redownload_tickers(['AAPL', 'MSFT', 'GOOGL'])

## 7️⃣ Verify Downloaded Data

### Check Files

In [21]:
# Count saved files
csv_files = list(DATA_DIR.glob("*.csv.gz"))
print(f"📁 Total files saved: {len(csv_files)}")

# Show statistics for sample tickers
sample_tickers = ['AAPL', 'MSFT', 'GOOGL', 'MTD']

print("\n📊 Sample Statistics:\n")
for ticker in sample_tickers:
    file_path = DATA_DIR / f"{ticker}.csv.gz"
    if file_path.exists():
        df = pd.read_csv(file_path, compression='gzip', parse_dates=['Datetime'])
        print(f"  {ticker}:")
        print(f"    • Records: {len(df)}")
        print(f"    • Period: {df['Datetime'].min()} to {df['Datetime'].max()}")
        print(f"    • File size: {file_path.stat().st_size / 1024:.2f} KB")
        print()
    else:
        print(f"  {ticker}: ❌ File not found\n")

📁 Total files saved: 1007

📊 Sample Statistics:

  AAPL:
    • Records: 26
    • Period: 2025-11-03 14:30:00+00:00 to 2025-11-04 20:30:00+00:00
    • File size: 0.87 KB

  MSFT:
    • Records: 26
    • Period: 2025-11-03 14:30:00+00:00 to 2025-11-04 20:30:00+00:00
    • File size: 0.87 KB

  GOOGL:
    • Records: 26
    • Period: 2025-11-03 14:30:00+00:00 to 2025-11-04 20:30:00+00:00
    • File size: 0.84 KB

  MTD:
    • Records: 26
    • Period: 2025-11-03 14:30:00+00:00 to 2025-11-04 20:30:00+00:00
    • File size: 0.80 KB



### View Sample Data

In [22]:
# Load and display sample data
ticker_example = 'BRK-B'
file_path = DATA_DIR / f"{ticker_example}.csv.gz"

if file_path.exists():
    df = pd.read_csv(file_path, compression='gzip', parse_dates=['Datetime'])
    
    print(f"📈 Sample data from {ticker_example}:\n")
    print(df.head(10))
    
    print(f"\n📊 Dataset info:")
    print(df.info())
else:
    print(f"❌ File {ticker_example}.csv.gz not found")

📈 Sample data from BRK-B:

                   Datetime       Close        High         Low        Open  \
0 2025-11-03 14:30:00+00:00  473.950012  479.989990  473.149994  479.510010   
1 2025-11-03 15:00:00+00:00  477.079987  477.489014  473.609985  474.019989   
2 2025-11-03 15:30:00+00:00  477.862305  478.878387  476.000000  477.140015   
3 2025-11-03 16:00:00+00:00  475.010010  477.855011  474.820007  477.850006   
4 2025-11-03 16:30:00+00:00  475.690002  476.714508  474.700012  475.019989   
5 2025-11-03 17:00:00+00:00  476.605011  477.000000  475.119995  475.690002   
6 2025-11-03 17:30:00+00:00  477.320007  477.769989  476.630005  476.690002   
7 2025-11-03 18:00:00+00:00  477.109985  477.440002  476.750000  477.290009   
8 2025-11-03 18:30:00+00:00  477.225006  478.029999  477.000000  477.165009   
9 2025-11-03 19:00:00+00:00  477.100006  477.359985  476.200012  477.269989   

    Volume Ticker  
0  1107179  BRK-B  
1   505015  BRK-B  
2   404784  BRK-B  
3   308967  BRK-B  
4  

## 8️⃣ Load Specific Ticker Data

Helper function to easily load any ticker's data

In [23]:
def load_ticker(ticker):
    """
    Load data for a specific ticker
    
    Args:
        ticker (str): Ticker symbol
    
    Returns:
        pd.DataFrame: Ticker data or None if not found
    """
    file_path = DATA_DIR / f"{ticker}.csv.gz"
    
    if file_path.exists():
        return pd.read_csv(file_path, compression='gzip', parse_dates=['Datetime'])
    else:
        print(f"❌ File {ticker}.csv.gz not found")
        return None

# Example usage
df_apple = load_ticker('DASH')
df_apple

,Datetime,Close,High,Low,Open,Volume,Ticker
0,2025-11-03 14:30:00+00:00,247.729996,257.064209,247.729996,255.899994,316608,DASH
1,2025-11-03 15:00:00+00:00,244.759995,248.039993,244.190002,247.925003,338951,DASH
2,2025-11-03 15:30:00+00:00,245.580002,246.800003,243.820007,244.875107,174541,DASH
3,2025-11-03 16:00:00+00:00,244.574997,245.809998,244.460007,245.580002,158834,DASH
4,2025-11-03 16:30:00+00:00,241.919998,245.110001,241.919998,244.574997,251911,DASH
5,2025-11-03 17:00:00+00:00,243.475006,245.149994,241.089996,241.669998,344838,DASH
6,2025-11-03 17:30:00+00:00,244.735001,245.020004,242.889999,243.365005,188333,DASH
7,2025-11-03 18:00:00+00:00,243.570007,245.080002,243.050003,244.649994,166823,DASH
8,2025-11-03 18:30:00+00:00,243.020004,244.145004,242.860001,243.524994,167577,DASH
9,2025-11-03 19:00:00+00:00,242.240005,243.226807,242.029999,243.110001,982791,DASH


---

## 📝 Daily Usage Instructions

### To download data every day:

1. **Open this notebook**
2. **Run all cells** from Section 1-5 (setup and configuration)
3. **Run Section 6 - Option B** (Full Download)
4. **Check Section 7** to verify the data

### Key Features:

- ✅ **Automatic date calculation** (3 days: from 2 days ago to today)
- ✅ **Incremental append** (new data added without overwriting)
- ✅ **Duplicate removal** (automatic)
- ✅ **Error logging** (check `download_log.txt`)
- ✅ **Compressed storage** (CSV.gz format)

### Notes:

- 📅 30-minute data is available for the **last 60 days** only
- 📊 Stocks have ~14 records/day (market hours only)
- 📁 Tickers are loaded from **tickers.csv** file
- 🕒 Download time depends on the number of tickers in the CSV file

---